In [1]:
import os
import warnings
import time
import json
import gzip
from lobsterpy.cohp.analyze import Analysis
from lobsterpy.cohp.describe import Description
import multiprocessing as mp
from multiprocessing.pool import ThreadPool
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
parent=os.getcwd()
os.chdir(parent)

In [3]:
os.chdir('Results/')

In [4]:
mpids= [f for f in os.listdir() if not f.startswith('t') and not f.startswith('.') and not f.startswith('__')
            and os.path.isdir(f)]
mats= list(set([ids.split('_')[0] for ids in mpids]))
mats.sort()

In [5]:
def get_lobsterpy_jsons(mpid, json_save_dir='/hpc-user/AG-JGeorge/anaik/Phonon_dataset_LSO/Lobsterpy_json/'):
    
    directory = mpid
    try:
        start = time.time()
        analyse = Analysis(
            path_to_poscar=os.path.join(directory, "POSCAR.gz"),
            path_to_icohplist=os.path.join(directory, "ICOHPLIST.lobster.gz"),
            path_to_cohpcar=os.path.join(directory, "COHPCAR.lobster.gz"),
            path_to_charge=os.path.join(directory, "CHARGE.lobster.gz"),
            path_to_madelung=os.path.join(directory, "MadelungEnergies.lobster.gz"),
            whichbonds="cation-anion",
            summed_spins=True,
        )
        cba_run_time = time.time() - start
    except (ValueError, KeyError):
        start = time.time()
        analyse = Analysis(
            path_to_poscar=os.path.join(directory, "POSCAR.gz"),
            path_to_icohplist=os.path.join(directory, "ICOHPLIST.lobster.gz"),
            path_to_cohpcar=os.path.join(directory, "COHPCAR.lobster.gz"),
            path_to_charge=os.path.join(directory, "CHARGE.lobster.gz"),
            path_to_madelung=os.path.join(directory, "MadelungEnergies.lobster.gz"),
            whichbonds="all",
            summed_spins=True,
        )
        cba_run_time = time.time() - start
    cba_cohp_plot_data = {}  # Initialize dict to store plot data
    for index, (_site, val) in enumerate(
        analyse.condensed_bonding_analysis["sites"].items()
    ):
        cohp_data = analyse.chemenv.completecohp.get_summed_cohp_by_label_list(
            val["relevant_bonds"]
        ).as_dict()
        spinup_cohps = cohp_data["COHP"]["1"]
        spindown_cohps = cohp_data["COHP"]["-1"]
        energies = cohp_data["energies"]
        efermi = cohp_data["efermi"]
        cba_cohp_plot_data.update(
            {
                analyse.set_labels_cohps[index][0]: {
                    "COHP_spin_up": spinup_cohps,
                    "COHP_spin_down": spindown_cohps,
                    "Energies": energies,
                    "Efermi": efermi,
                }
            }
        )
    describe = Description(analysis_object=analyse)

    result= analyse.condensed_bonding_analysis

    result.update({'cohp_plot_data':cba_cohp_plot_data, 
                   'text_description':" ".join(describe.text),
                   'lobsterpy_run_time': cba_run_time
                  })
    
    with gzip.open(os.path.join(json_save_dir, "{}.json.gz".format(mpid)), 'wt', encoding='UTF-8') as f:
        json.dump(result, f)


    return mpid+' Done'

In [ ]:
with mp.Pool(processes=4,maxtasksperchild=1) as pool:
    results = tqdm(
        pool.imap_unordered(get_lobsterpy_jsons, mats, chunksize=1),
        total=len(mats),
    )
    row=[]
    for result in results:
        row.append(result)

  0%|                                                  | 0/1520 [00:00<?, ?it/s]